This Jupyter Notebook accompanies the paper "Mukai lifting of self-dual points" by Barbara Betti and Leonie Kayser.
The code is in Julia language.
The folder $\texttt{MukaiLiftSource}$ contains all the files and we need to work inside this directory.
We describe how to use the package $\texttt{MukaiLiftP6}$.

In [1]:
using Pkg;
Pkg.activate("MukaiLiftP6")

  Activating project at `~/Documents/GitHub/Gr26/MukaiLiftSource/MukaiLiftP6`


In [ ]:
using MukaiLiftP6
using LinearAlgebra
using HomotopyContinuation

We define the configuration of points as a $7\times 14$ matrix $\Gamma$. 
We prove that it is self-dual by computing the matrix $\Lambda$ satisfying condition $(ii)$ of Lemma $2.1$ and checking that $\Gamma\cdot\Lambda\cdot\Gamma^\intercal=0$.

In [ ]:
Gamma = [7 -2 6 -1 -6 1 -9 7 0 6 1 8 -3 7; -1 2 -5 -2 0 -4 3 -3 -4 -3 4 -2 4 -1; 1 4 -1 -5 -3 6 8 -1 -8 -3 5 1 -6 -8; 3 -6 4 -3 -4 6 0 5 8 2 3 2 -8 0; 1 -2 1 0 -4 2 2 3 4 -1 2 2 -2 -2; 0 -6 -5 6 3 7 -3 2 8 -7 -6 -3 -5 5; -3 3 -4 1 4 3 2 -3 -6 -4 -3 -4 -1 -2]
Lambda = certify_selfdual(Gamma)
norm( Gamma*Lambda*transpose(Gamma), Inf) 

We compute the Orthogonal normal form $\Gamma_{\text{ONF}}= [ I | P]$ of $\Gamma$ together with the linear projective transformation $A$ such that $A\cdot \Gamma_{\text{ONF}}= \Gamma\cdot \Lambda_{\text{scale}}$.

In [ ]:
Gamma_ONF, A, Lambda_scale = orthogonal_normal_form(Gamma)
norm(A*Gamma_ONF - Gamma*Lambda_scale, Inf)

The orthogonal normal form of $\Gamma$ is given by $[I | P]$, where $P$ is the orthogonal matrix:

In [ ]:
P = Matrix{Float64}(Gamma_ONF[:,8:14])

We compute the Telen norma form $\Gamma_{\text{SNF}}$ of $\Gamma$ using the Cayley transform of $P$.

In [ ]:
S = cayley(P)
Gamma_SNF = [I+S I-S]

The target parameters $S_{\text{target}}$ of the homotopy are the entries of the skew-symmetric matrix $S$.

In [ ]:
S_target = skew_to_vector(S)

We solve the mukai lifting problem for $\Gamma$.

In [ ]:
@time result = HomotopyContinuation.solve(parametrized_system, l_start; compile = false, start_parameters=S_start, target_parameters=S_target)

We analyze the solution. There is one path tracked with $1429$ steps. 
The residual is the infinity norm of the vector with the equations evaluated in the computed solution and it is numerically zero.

In [ ]:
sol = solutions(result)[1]
result.path_results

The embedding $\widehat L: \mathbb{P}^6\simeq \mathbb{L}\subseteq \mathbb{P}^{14} $ such that $\widehat L(\Gamma_{\text{SNF}})=\mathbb{L}\cap X_8$ is 

In [ ]:
L_hat = L_start + sum(sol[i]*A_rand[i] for i in eachindex(l))

Composing with the previous coordinate transformations $\Gamma \leadsto \Gamma_{\text{SNF}}$ we obtain $L \colon \mathbb{P}^6 \to \mathbb{P}^{14}$ such that $L(\Gamma)= \mathbb{L}\cap X_8$

In [ ]:
L = L_hat*(I+S)*inv(A)

We can verify $L(\Gamma) = \mathbb{L} \cap X_8$ by evaluating $ L(\gamma_i)$ in the Plücker relations.

In [ ]:
q = poly_to_func(plück_oscar)
maximum([ norm(q(L*Gamma[:,i]) ,Inf) for i =1:14])